# Import model

In [ ]:
'''
model_path = "./fine_tuned_Roberta_imdb"

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

'''

c:\Users\Lenovo\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("DeepAxion/distilbert-imdb-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("DeepAxion/distilbert-imdb-sentiment")

c:\Users\Lenovo\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Lenovo\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\models--DeepAxion--distilbert-imdb-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer

In [2]:
import pandas as pd
df = pd.read_csv("C:/Users/Lenovo/Desktop/NLP/Final_project/unseen_review/review.csv")

df.head(20)

,sentiment,review
0,Negative,I had no background knowledge of this movie be...
1,Negative,I am a huge Jane Austen fan and I ordered the ...
2,Negative,Nothing to say but Wow! Has anyone actually ha...
3,Negative,i like Jane Austin novels. I love Pride and Pr...
4,Negative,In this day and age of incredible special movi...
5,Negative,Becky Harris plays the female shopper whose mi...
6,Negative,Watching TRUTH ABOUT LOVE (is this a double en...
7,Negative,"A painfully protracted, maudlin and predictabl..."
8,Negative,"Let's be honest. As a film school project, mad..."
9,Negative,A terrible movie as everyone has said. What ma...


In [3]:
import pandas as pd
import torch
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm  # progress bar

BATCH_SIZE = 8  # ✅ ปรับตาม VRAM

# ✅ เลือกอุปกรณ์ (GPU ถ้ามี, ถ้าไม่มีใช้ CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

# ✅ ย้ายโมเดลไป GPU
model = model.to(device)

# ✅ โหลด CSV
df = pd.read_csv("C:/Users/Lenovo/Desktop/NLP/Final_project/unseen_review/review.csv")


all_pred_labels = []

# ✅ predict แบบ batch
for i in tqdm(range(0, len(df), BATCH_SIZE)):
    batch_texts = df["review"].iloc[i:i+BATCH_SIZE].tolist()
    
    # ✅ tokenize + ส่งไป GPU
    inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to(device)
    
    # ✅ predict
    with torch.no_grad():  
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        batch_preds = probs.argmax(axis=1).cpu().numpy()  # ✅ กลับ CPU เพื่อเก็บ
    
    all_pred_labels.extend(batch_preds)

# ✅ เพิ่มผลลัพธ์เข้า DataFrame
df["predicted_label"] = all_pred_labels
df["predicted_sentiment"] = ["positive" if p == 1 else "negative" for p in all_pred_labels]

# ✅ แปลง label จริงเป็น 0/1
if "sentiment" in df.columns:  # ถ้ามี label จริง
    if df["sentiment"].dtype == object:  
        df["true_label"] = df["sentiment"].apply(lambda x: 1 if x.lower()=="positive" else 0)
    else:
        df["true_label"] = df["sentiment"]

    # ✅ คำนวณ Accuracy
    acc = accuracy_score(df["true_label"], df["predicted_label"])
    print(f"\n Accuracy: {acc*100:.2f}%")

    # ✅ Precision / Recall / F1
    print("\n=== Classification Report ===")
    print(classification_report(df["true_label"], df["predicted_label"], target_names=["negative", "positive"]))
else:
    print("\n No ground-truth labels found, skipping accuracy calculation")

# ✅ แสดงตัวอย่าง 10 row แรก
print("\n=== Sample Predictions (first 10) ===")
for _, row in df.head(10).iterrows():
    print(f"Review: {row['review'][:80]}...")
    if "sentiment" in df.columns:
        print(f"  ✅ True: {row['sentiment']} | 🔮 Predicted: {row['predicted_sentiment']}\n")
    else:
        print(f"  🔮 Predicted: {row['predicted_sentiment']}\n")

# ✅ บันทึกผลลัพธ์
df.to_csv("C:/Users/Lenovo/Desktop/NLP/Final_project/prediction_result/reviews_with_predictions_distilbert-imdb-sentiment_model_full.csv", index=False)
print("\n✅ Saved predictions to reviews_with_predictions_distilbert-imdb-sentiment_model_full.csv")


✅ Using device: cuda


100%|██████████| 1250/1250 [01:45<00:00, 11.83it/s]



 Accuracy: 97.88%

=== Classification Report ===
              precision    recall  f1-score   support

    negative       0.98      0.98      0.98      5081
    positive       0.98      0.98      0.98      4919

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000


=== Sample Predictions (first 10) ===
Review: I had no background knowledge of this movie before I bought it, but it sounded c...
  ✅ True: Negative | 🔮 Predicted: negative

Review: I am a huge Jane Austen fan and I ordered the movie from Amazon.UK just so I cou...
  ✅ True: Negative | 🔮 Predicted: negative

Review: Nothing to say but Wow! Has anyone actually had somebody sneak up on them in an ...
  ✅ True: Negative | 🔮 Predicted: negative

Review: i like Jane Austin novels. I love Pride and Prejudice and Sense and Sensibility ...
  ✅ True: Negative | 🔮 Predicted: negative

Review: In this day and age of incredibl